In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

DAYS_IN_YEAR = 252
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


def get_ticker_from_yahoo(ticker: str, start: datetime, end: datetime):
    """ Get ticker from Yahoo Finance.
    """
    return yf.Ticker(ticker).history(start=start, end=end)


def get_ticker_data(ticker: str, start: datetime, end: datetime):
    """ Get the details stats for a stock ticker.
    
    Args:
        ticker (str): the ticker symbol on Yahoo Finance.
        start (datetime): start time of data.
        end (datatime): end time of data.

    Returns:
        Dataframe: contains the daily stats of the ticker.
    """
    data = get_ticker_from_yahoo(ticker, start, end)
    data['Change'] = data['Close'].pct_change()
    data['Gross Return'] = data['Change'] + 1
    data['Log Return'] = np.log(data['Gross Return'])
    return data


def get_risk_free_return(start: datetime, end: datetime):
    """ Get risk-free return.

    The risk-free return is yielded by 13 Week Treasuray Bill.
    Note that the data from Yahoo Finance represent is annualized return.
    Each daily value contains annualized return %.

    Args:
        start (datetime): start time of data.
        end (datatime): end time of data.

    Returns:
        Dataframe: contains the daily stats of risk-free return.
    """
    irx = get_ticker_from_yahoo('^IRX', start ,end)
    irx['Annaul Log Return'] = np.log(1 + (irx['Close'] / 100)) # divide by 100 since it's %
    irx['Log Return'] = irx['Annaul Log Return'] / DAYS_IN_YEAR
    return irx


def get_borrow_rate(start, end):
    """ Get borrow rate.

    Assume that borrowing costs are equal to the risk-free rate plus 120 basis points.

    Args:
        start (datetime): start time of data.
        end (datatime): end time of data.

    Returns:
        Dataframe: contains the daily stats of borrow rate.
    """
    irx = get_ticker_from_yahoo('^IRX', start ,end)
    irx['Borrow Rate'] = np.log(1 + ((irx['Close'] + 1.2) / 100))  / DAYS_IN_YEAR # divide by 100 since it's %
    return irx


def get_sharp_ratio(ticker: str, start: datetime, end: datetime):
    """ Get sharp ratio for a stock ticker.
    
    Args:
        ticker (str): the ticker symbol on Yahoo Finance.
        start (datetime): start time of data.
        end (datatime): end time of data.

    Returns:
        float: the sharp ratio of the ticker data.
    """
    ticker_data = get_ticker_data(ticker, start, end)
    irx_data = get_risk_free_return(start, end)
    ticker_data['Excess Return'] = ticker_data['Log Return'] - irx_data['Log Return']

    excess_return_mean = np.mean(ticker_data['Excess Return'])
    excess_return_std = np.std(ticker_data['Excess Return'])
    return excess_return_mean / excess_return_std


def get_expect_log_return(ticker: str, start: datetime, end: datetime):
    """ Get expected log return and standard deviation for a stock ticker.
    
    Args:
        ticker (str): the ticker symbol on Yahoo Finance.
        start (datetime): start time of data.
        end (datatime): end time of data.

    Returns:
        float: the expected log return of the stock.
        float: the standard deviation of log return.
    """

    ticker_data = get_ticker_data(ticker, start, end)
    return (np.mean(ticker_data['Log Return']), np.std(ticker_data['Log Return']))


## Index ETFs Sharp Ratio Analysis

Calculate the Sharpe ratio of each ETF and compare it with the Sharpe ratio for the S&P 500 index (symbol
ˆGSPC).

In [27]:


tickers = ['BETZ', 'VGT', 'IBUY', 'ARKK', 'ARKG', 'ARKW', 'ARKQ', 'ARKF', 'CLOU', 'ESPO', 'XBI', 'XLV', 'XPH', 'XLP', 'XLY', 'XRT', 'ITB', 'XLF', 'IYT', 'ITA', 'XOP', 'XLE', 'VNQ', 'XLB', 'TAN', 'XME', 'JETS']
start = datetime(2020, 4, 9)
end = datetime(2020, 9, 23)

sharp_sp500 = get_sharp_ratio('^GSPC', start, end)
datas = []
for ticker in tickers:
    data = {}
    data['ticker'] = ticker
    data['sharp'] = get_sharp_ratio(ticker, start, end)
    data['relative_sharp'] = data['sharp'] - sharp_sp500
    datas.append(data)

print(f'{"SP500":6} sharp: {sharp_sp500:.6f}')
datas.sort(key= lambda data: data['relative_sharp'], reverse=True)
for data in datas:
    print(f'{data["ticker"]:6} sharp: {data["sharp"]:.6f} || relative_sharp: {data["relative_sharp"]:.6f}')



SP500  sharp: 0.115277
IBUY   sharp: 0.312674 || relative_sharp: 0.197397
ARKF   sharp: 0.280109 || relative_sharp: 0.164832
ARKW   sharp: 0.255015 || relative_sharp: 0.139738
ESPO   sharp: 0.241484 || relative_sharp: 0.126207
TAN    sharp: 0.237422 || relative_sharp: 0.122145
ARKK   sharp: 0.235704 || relative_sharp: 0.120427
ARKG   sharp: 0.201813 || relative_sharp: 0.086536
XRT    sharp: 0.200940 || relative_sharp: 0.085663
ARKQ   sharp: 0.199834 || relative_sharp: 0.084557
XLY    sharp: 0.193831 || relative_sharp: 0.078554
CLOU   sharp: 0.180409 || relative_sharp: 0.065132
ITB    sharp: 0.173852 || relative_sharp: 0.058575
VGT    sharp: 0.153796 || relative_sharp: 0.038519
IYT    sharp: 0.140279 || relative_sharp: 0.025002
BETZ   sharp: 0.135008 || relative_sharp: 0.019731
XBI    sharp: 0.126391 || relative_sharp: 0.011114
XLB    sharp: 0.125031 || relative_sharp: 0.009754
XME    sharp: 0.101942 || relative_sharp: -0.013335
XPH    sharp: 0.097488 || relative_sharp: -0.017789
XLP   

## Bonds ETFs vs. SP500
Calculate the expectation and standard deviation of the returns for chosen bond ETF. Compare those with the S&P 500 index. Also, compare Sharpe ratio too.

In [23]:
tickers = ['^GSPC', 'NUAG', 'USTB', 'CBON', 'BND', 'VCSH', 'TIP', 'VCIT', 'HYLB', 'KCNY', 'IUSB', 'HYLD', 'HYXU', 'NEAR', 'IGLB', 'LQD', 'SHY', 'IEF', 'TLT', 'ICVT', 'CWB', 'PFF']
periods = [
    (datetime(2010, 8, 31), datetime(2020, 9, 1)),
    (datetime(2015, 8, 31), datetime(2020, 9, 1)),
]

for start, end in periods:
    print(f'\nGetting data {start} -> {end}')

    datas = []
    for ticker in tickers:
        print(f'{ticker} ', end='')
        data = {}
        data['ticker'] = ticker
        data['mean'], data['std'] = get_expect_log_return(ticker, start, end)
        data['sharp'] = get_sharp_ratio(ticker, start, end)
        datas.append(data)
    
    print('\nSorted by sharp ratio:')
    datas.sort(key= lambda data: data['sharp'], reverse=True)
    for data in datas:
        color_prefix = '\x1b[94m' if data['ticker'] == '^GSPC' else ''
        color_postfix = '\x1b[0m' if data['ticker'] == '^GSPC' else ''
        print(f'{color_prefix}{data["ticker"]:6} mean: {data["mean"]*100:.6f}%  std: {data["std"]*100:.6f}%' +
              f'  sharp: {data["sharp"]:.6f}{color_postfix}')
    print('---------------------------------------------------------')



Getting data 2010-08-31 00:00:00 -> 2020-09-01 00:00:00
^GSPC NUAG USTB CBON BND VCSH TIP VCIT HYLB KCNY IUSB HYLD HYXU NEAR IGLB LQD SHY IEF TLT ICVT CWB PFF 
Sorted by sharp ratio:
USTB   mean: 0.013769%  std: 0.138767%  sharp: 0.055469
VCIT   mean: 0.020879%  std: 0.341645%  sharp: 0.054505
CWB    mean: 0.044660%  std: 0.797649%  sharp: 0.052718
VCSH   mean: 0.011651%  std: 0.188412%  sharp: 0.050389
ICVT   mean: 0.050866%  std: 1.028362%  sharp: 0.045993
IUSB   mean: 0.015409%  std: 0.261715%  sharp: 0.045586
BND    mean: 0.013945%  std: 0.270118%  sharp: 0.044684
LQD    mean: 0.021567%  std: 0.457141%  sharp: 0.042864
^GSPC  mean: 0.047859%  std: 1.092130%  sharp: 0.042609
SHY    mean: 0.004609%  std: 0.058542%  sharp: 0.039832
IEF    mean: 0.016199%  std: 0.382890%  sharp: 0.037710
TIP    mean: 0.013923%  std: 0.349785%  sharp: 0.034819
IGLB   mean: 0.026119%  std: 0.702596%  sharp: 0.034132
NUAG   mean: 0.016031%  std: 0.317004%  sharp: 0.032778
TLT    mean: 0.027242%  std: 0.9

In [25]:
sp500_10yr_sharp = get_sharp_ratio('^GSPC', datetime(2010, 8, 31), datetime(2020, 9, 1))
sp500_5yr_sharp = get_sharp_ratio('^GSPC', datetime(2015, 8, 31), datetime(2020, 9, 1))
print(f'sp500 sharp 5yr/10yr = {sp500_5yr_sharp / sp500_10yr_sharp}')

sp500 sharp 5yr/10yr = 0.8488676576428935


# Short Selling
Calculate the returns for asset over 2 month windows by using its price on a given day and its price 42 days prior. Note that for each day of
returns, there is a different two month window behind it. Also analyze:
-  The mean return of a asset over all of the 2 month windows.
-  The percentage of those 2 month windows yield negative returns.

In [61]:
def get_log_return_sliding_window(ticker: str, start: datetime, end: datetime, window_size: int):
    """ 
    Get log return for each time window. For window size = x, an entry will contain the log return from date d - x to d. 
    Next entry contains d + 1 - x to d + 1.
    
    Args:
        ticker (str): the ticker symbol on Yahoo Finance.
        start (datetime): start time of data.
        end (datatime): end time of data.
        window_size (int): the size of the window.

    Returns:
        Dataframe: contains the log return in each window.
    """
    ticker_data = get_ticker_data(ticker, start, end)
    ticker_data['Rolling Start Date'] = ticker_data.index.shift(periods=-1 * (window_size), freq='B')
    ticker_data['Rolling End Date'] = ticker_data.index
    ticker_data['Rolling Log Return'] = ticker_data['Log Return'].rolling(window_size).sum()
    ticker_data.dropna(subset=['Rolling Log Return'], inplace=True)
    return ticker_data

tickers = ['^GSPC', 'IVV', 'QQQ', 'IJR', 'IJH', 'SHY', 'IEF', 'TLT', 'JETS', 'VGT', 'CLOU', 'XBI', 'XPH', 'XLV', 'XLP', 'XLY', 'XRT', 'ITB', 'XLF', 'KRE', 'XLI', 'IYT', 'ITA', 'XLE', 'XOP', 'VNQ', 'XLU', 'XLB', 'XME', 'DBA', 'CORN', 'SOYB', 'WEAT']
start = datetime(2010, 8, 31)
end = datetime(2020, 9, 1)
window_size = 42
datas = []

print(f'\nGetting data {start} -> {end}')
for ticker in tickers:
    print(f'{ticker} ', end='')
    data = {}
    data['ticker'] = ticker

    ticker_data = get_log_return_sliding_window(ticker, start, end, window_size)
    data['rolling_mean'] = np.mean(ticker_data['Rolling Log Return'])

    rolling_neg_cnt = (ticker_data['Rolling Log Return'] < 0).sum()
    data['rolling_neg_pct'] =  rolling_neg_cnt / ticker_data['Rolling Log Return'].size * 100
    datas.append(data)

print('\nSorted by the percentage of negative log returns in all windows:')
datas.sort(key= lambda data: data['rolling_neg_pct'], reverse=True)
for data in datas:
    color_prefix = '\x1b[94m' if data['ticker'] == '^GSPC' else ''
    color_postfix = '\x1b[0m' if data['ticker'] == '^GSPC' else ''
    print(f'{color_prefix}{data["ticker"]:6} rolling_mean: {data["rolling_mean"]:.6f}%'+ 
            f' rolling_neg_pct: {data["rolling_neg_pct"]:.6f}%{color_postfix}')
print('---------------------------------------------------------')


Getting data 2010-08-31 00:00:00 -&gt; 2020-09-01 00:00:00
^GSPC IVV QQQ IJR IJH SHY IEF TLT JETS VGT CLOU XBI XPH XLV XLP XLY XRT ITB XLF KRE XLI IYT ITA XLE XOP VNQ XLU XLB XME DBA CORN SOYB WEAT 
Sorted by the percentage of negative log returns in all windows:
WEAT   rolling_mean: -0.028027% rolling_neg_pct: 68.430574%
DBA    rolling_mean: -0.011147% rolling_neg_pct: 64.352039%
CORN   rolling_mean: -0.017147% rolling_neg_pct: 62.858296%
SOYB   rolling_mean: -0.008661% rolling_neg_pct: 57.123474%
XME    rolling_mean: -0.011334% rolling_neg_pct: 52.644328%
JETS   rolling_mean: -0.009797% rolling_neg_pct: 47.582502%
XOP    rolling_mean: -0.017997% rolling_neg_pct: 46.992329%
XLE    rolling_mean: -0.002094% rolling_neg_pct: 44.610416%
TLT    rolling_mean: 0.012630% rolling_neg_pct: 38.998789%
KRE    rolling_mean: 0.011924% rolling_neg_pct: 38.635446%
XRT    rolling_mean: 0.016596% rolling_neg_pct: 38.070246%
XPH    rolling_mean: 0.015370% rolling_neg_pct: 36.415018%
IYT    rolling_mean

# Leverage
Consider a 4:1 leveraged investment in the security and calculate the mean, standard deviation,
and Sharpe ratio for that leveraged investment over the past 2 years.

- Assume that your borrowing costs are equal to the risk-free rate plus 120 “basis
points.” A basis point is a financial term that represents 1/100 of one percent. So,
in other words, if the annual risk-free rate is 2.5%, then this + 120 basis points
(bps) would be 3.7%. This borrowing cost is based on a real example for a large
institutional investor.

-  Given this borrowing rate, calculate the daily log returns on the 4:1 leveraged portfolio.

-  Compute the mean, standard deviation, and Sharpe ratio for this leveraged investment. How do they compare with these statistics for the un-leveraged investment
over the same period?



In [5]:
tickers = ['^GSPC', 'IVV', 'QQQ', 'IJR', 'IJH', 'SHY', 'IEF', 'TLT', 'JETS', 'VGT', 'CLOU', 'XBI', 'XPH', 'XLV', 'XLP', 'XLY', 'XRT', 'ITB', 'XLF', 'KRE', 'XLI', 'IYT', 'ITA', 'XLE', 'XOP', 'VNQ', 'XLU', 'XLB', 'XME', 'DBA', 'CORN', 'SOYB', 'WEAT']
start = datetime(2018, 10, 10)
end = datetime(2020, 10, 11)
datas = []

print(f'\nGetting data {start} -> {end}')
for ticker in tickers:
    print(f'{ticker} ', end='')
    data = {}
    data['ticker'] = ticker
    data['mean'], data['std'] = get_expect_log_return(ticker, start, end)
    data['sharp'] = get_sharp_ratio(ticker, start, end)
    datas.append(data)

print('\nSorted by sharp ratio:')
datas.sort(key= lambda data: data['sharp'], reverse=True)
for data in datas:
    color_prefix = '\x1b[94m' if data['ticker'] == '^GSPC' else ''
    color_postfix = '\x1b[0m' if data['ticker'] == '^GSPC' else ''
    print(f'{color_prefix}{data["ticker"]:6} mean: {data["mean"]*100:.6f}%  std: {data["std"]*100:.6f}%' +
            f'  sharp: {data["sharp"]:.6f}{color_postfix}')
print('---------------------------------------------------------')


print(f'\nGetting 4:1 leveraged data {start} -> {end}')
datas.clear()
borrow_rate = get_borrow_rate(start, end)
for ticker in tickers:
    print(f'{ticker} ', end='')
    data = {}
    data['ticker'] = ticker

    ticker_data = get_ticker_data(ticker, start, end)
    ticker_data['Excess Return'] = 4 * ticker_data['Log Return'] - 3 * borrow_rate['Borrow Rate']

    data['mean'], data['std'] = (np.mean(ticker_data['Excess Return']), np.std(ticker_data['Excess Return']))
    data['sharp'] = data['mean'] / data['std']
    datas.append(data)


print('\nSorted by sharp ratio:')
datas.sort(key= lambda data: data['sharp'], reverse=True)
for data in datas:
    color_prefix = '\x1b[94m' if data['ticker'] == '^GSPC' else ''
    color_postfix = '\x1b[0m' if data['ticker'] == '^GSPC' else ''
    print(f'{color_prefix}{data["ticker"]:6} mean: {data["mean"]*100:.6f}%  std: {data["std"]*100:.6f}%' +
            f'  sharp: {data["sharp"]:.6f}{color_postfix}')
print('---------------------------------------------------------')



Getting data 2018-10-10 00:00:00 -> 2020-10-11 00:00:00
^GSPC IVV QQQ IJR IJH SHY IEF TLT JETS VGT CLOU XBI XPH XLV XLP XLY XRT ITB XLF KRE XLI IYT ITA XLE XOP VNQ XLU XLB XME DBA CORN SOYB WEAT 
Sorted by sharp ratio:
SHY    mean: 0.015068%  std: 0.078735%  sharp: 0.118181
IEF    mean: 0.045133%  std: 0.395094%  sharp: 0.097811
CLOU   mean: 0.143021%  std: 2.053841%  sharp: 0.066919
TLT    mean: 0.076049%  std: 1.077496%  sharp: 0.063227
VGT    mean: 0.105020%  std: 2.031529%  sharp: 0.052235
QQQ    mean: 0.095039%  std: 1.847045%  sharp: 0.051916
ITB    mean: 0.117372%  std: 2.581462%  sharp: 0.044079
XLY    mean: 0.067330%  std: 1.724949%  sharp: 0.038267
XLP    mean: 0.049868%  std: 1.356419%  sharp: 0.034876
IVV    mean: 0.045023%  std: 1.694545%  sharp: 0.026830
XLU    mean: 0.044095%  std: 1.793120%  sharp: 0.025620
XLB    mean: 0.043932%  std: 1.869289%  sharp: 0.024383
^GSPC  mean: 0.037361%  std: 1.693221%  sharp: 0.022548
XBI    mean: 0.061254%  std: 2.306989%  sharp: 0.022

In [5]:
data = pd.Series([1, 3, 8])
dataframe = pd.DataFrame({'A': [1,2,3,4], 'B': [3,3,3,3]})
dataframe2 = pd.DataFrame({'A': [1,2,3,4,5,6,7,8], 'B': [3,3,3,3,6,6,6,6]})
print (dataframe['A'] - 1)

0    0
1    1
2    2
3    3
Name: A, dtype: int64


In [47]:

df = pd.DataFrame(np.random.randint(-5, 5, size=(6, 4)), index=pd.date_range('20130108', periods=6), columns=list('ABCD'))
# df2 = pd.DataFrame(np.zeros((20, 4)), index=pd.date_range('20130101', periods=20), columns=list('ABCD'))

print (df)

pct = (df['A'] < 0).sum()
print(pct)
print(df['A'].size)

            A  B  C  D
2013-01-08 -2 -2 -5 -2
2013-01-09 -1  3  2  0
2013-01-10 -1  4 -4 -3
2013-01-11  4  0  1  4
2013-01-12 -1  4 -5  0
2013-01-13  2 -4  1  2
4
6
